In [3]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------------------
# DEFINIR EL MODELO
# -------------------------------
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(21,)),  # ← 21 features reales
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Salida continua para regresión
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)

# -------------------------------
# ENTRENAMIENTO POR LOTES
# -------------------------------
train_folder = "../data_subsets/train_test"
chunk_count = 56   # Número de bloques de entrenamiento
epochs = 10        # Épocas globales
batch_size = 256   # Tamaño de batch interno

print("🚀 Entrenando el modelo por lotes...")

for epoch in range(epochs):
    print(f"\n🌀 Época {epoch+1}/{epochs}")
    for i in range(chunk_count):
        try:
            X_path = os.path.join(train_folder, f"X_train_{i}.npy")
            y_path = os.path.join(train_folder, f"y_train_{i}.npy")

            X_train = np.load(X_path)
            y_train = np.load(y_path)

            print(f"  ➤ Entrenando con chunk {i}... ({X_train.shape[0]} muestras)", end=' ')
            model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)
            print("✔️")

            del X_train, y_train

        except FileNotFoundError:
            print(f"⚠️ Chunk {i} no encontrado, saltando.")
            continue

# -------------------------------
# EVALUACIÓN
# -------------------------------
try:
    X_test = np.load(os.path.join(train_folder, "X_test_0.npy"))
    y_test = np.load(os.path.join(train_folder, "y_test_0.npy"))
    print("\n📊 Evaluando modelo en test set...")
    loss, mae = model.evaluate(X_test, y_test)
    print(f"🔍 Loss: {loss:.4f} | MAE: {mae:.4f}")
except FileNotFoundError:
    print("⚠️ No se pudo evaluar el modelo. ¿Falta X_test_0.npy o y_test_0.npy?")

# -------------------------------
# GUARDAR EL MODELO
# -------------------------------
model_path = "../models/flight_price_predictor.h5"
model.save(model_path)
print(f"✅ Modelo guardado en: {model_path}")


🚀 Entrenando el modelo por lotes...

🌀 Época 1/10
  ➤ Entrenando con chunk 0... (838860 muestras) ✔️
  ➤ Entrenando con chunk 1... (838860 muestras) ✔️
  ➤ Entrenando con chunk 2... (838860 muestras) ✔️
  ➤ Entrenando con chunk 3... (838860 muestras) ✔️
  ➤ Entrenando con chunk 4... (838860 muestras) ✔️
  ➤ Entrenando con chunk 5... (838860 muestras) ✔️
  ➤ Entrenando con chunk 6... (838860 muestras) ✔️
  ➤ Entrenando con chunk 7... (838860 muestras) ✔️
  ➤ Entrenando con chunk 8... (838860 muestras) ✔️
  ➤ Entrenando con chunk 9... (838860 muestras) ✔️
  ➤ Entrenando con chunk 10... (838860 muestras) ✔️
  ➤ Entrenando con chunk 11... (838860 muestras) ✔️
  ➤ Entrenando con chunk 12... (838860 muestras) ✔️
  ➤ Entrenando con chunk 13... (838860 muestras) ✔️
  ➤ Entrenando con chunk 14... (838860 muestras) ✔️
  ➤ Entrenando con chunk 15... (838860 muestras) ✔️
  ➤ Entrenando con chunk 16... (838860 muestras) ✔️
  ➤ Entrenando con chunk 17... (838860 muestras) ✔️
  ➤ Entrenando con chunk

🔍 Loss: nan | MAE: nan
✅ Modelo guardado en: ../models/flight_price_predictor.h5


In [5]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------------------
# DEFINICIÓN DEL MODELO
# -------------------------------
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(21,)),  # Ajustado a tu número de features
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=optimizer,
    loss='mean_squared_error',
    metrics=['mae']
)

# -------------------------------
# ENTRENAMIENTO POR LOTES
# -------------------------------
train_folder = "../data_subsets/train_test"
chunk_count = 56   # Ajusta al número real de bloques
epochs = 10        # Número de épocas completas
batch_size = 256   # Tamaño de mini-batch

print("🚀 Entrenando el modelo por lotes...")

for epoch in range(epochs):
    print(f"\n🌀 Época {epoch+1}/{epochs}")
    for i in range(chunk_count):
        try:
            X_path = os.path.join(train_folder, f"X_train_{i}.npy")
            y_path = os.path.join(train_folder, f"y_train_{i}.npy")

            X_train = np.load(X_path)
            y_train = np.load(y_path)

            # COMPROBACIÓN DE NaNs
            if np.isnan(X_train).any() or np.isnan(y_train).any():
                print(f"❌ Chunk {i} tiene NaNs, lo saltamos.")
                continue

            # ESCALADO DE y_train
            y_train = y_train / 1000.0

            print(f"  ➤ Entrenando con chunk {i}... ({X_train.shape[0]} muestras)", end=' ')
            model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)
            print("✔️")

            del X_train, y_train

        except FileNotFoundError:
            print(f"⚠️ Chunk {i} no encontrado, saltando.")
            continue

# -------------------------------
# EVALUACIÓN
# -------------------------------
try:
    X_test = np.load(os.path.join(train_folder, "X_test_0.npy"))
    y_test = np.load(os.path.join(train_folder, "y_test_0.npy"))

    if np.isnan(X_test).any() or np.isnan(y_test).any():
        raise ValueError("❌ El set de test contiene NaNs.")

    y_test = y_test / 1000.0  # Escalar test igual que train

    print("\n📊 Evaluando modelo en test set...")
    loss, mae = model.evaluate(X_test, y_test)
    print(f"🔍 Loss: {loss:.4f} | MAE: {mae:.4f}")

except Exception as e:
    print(f"⚠️ No se pudo evaluar el modelo: {e}")

# -------------------------------
# GUARDAR MODELO ENTRENADO
# -------------------------------
model_path = "../models/flight_price_predictor_v2_scaled_lr005.h5"
model.save(model_path)
print(f"✅ Modelo guardado en: {model_path}")


🚀 Entrenando el modelo por lotes...

🌀 Época 1/10
  ➤ Entrenando con chunk 0... (762107 muestras) ✔️
  ➤ Entrenando con chunk 1... (759244 muestras) ✔️
  ➤ Entrenando con chunk 2... (735888 muestras) ✔️
  ➤ Entrenando con chunk 3... (735300 muestras) ✔️
  ➤ Entrenando con chunk 4... (735871 muestras) ✔️
  ➤ Entrenando con chunk 5... (737430 muestras) ✔️
  ➤ Entrenando con chunk 6... (736117 muestras) ✔️
  ➤ Entrenando con chunk 7... (736203 muestras) ✔️
  ➤ Entrenando con chunk 8... (734632 muestras) ✔️
  ➤ Entrenando con chunk 9... (737012 muestras) ✔️
  ➤ Entrenando con chunk 10... (738777 muestras) ✔️
  ➤ Entrenando con chunk 11... (740420 muestras) ✔️
  ➤ Entrenando con chunk 12... (755498 muestras) ✔️
  ➤ Entrenando con chunk 13... (734076 muestras) ✔️
  ➤ Entrenando con chunk 14... (615153 muestras) ✔️
  ➤ Entrenando con chunk 15... (714638 muestras) ✔️
  ➤ Entrenando con chunk 16... (746610 muestras) ✔️
  ➤ Entrenando con chunk 17... (745083 muestras) ✔️
  ➤ Entrenando con chunk

🔍 Loss: 0.0000 | MAE: 0.0012
✅ Modelo guardado en: ../models/flight_price_predictor_v2_scaled_lr005.h5


In [6]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------------------
# DEFINICIÓN DEL MODELO
# -------------------------------
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(21,)),  # Ajustado a tu número de features
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=optimizer,
    loss='mean_squared_error',
    metrics=['mae']
)

# -------------------------------
# ENTRENAMIENTO POR LOTES
# -------------------------------
train_folder = "../data_subsets/train_test"
chunk_count = 56   # Ajusta al número real de bloques
epochs = 10        # Número de épocas completas
batch_size = 256   # Tamaño de mini-batch

print("🚀 Entrenando el modelo por lotes...")

for epoch in range(epochs):
    print(f"\n🌀 Época {epoch+1}/{epochs}")
    for i in range(chunk_count):
        try:
            X_path = os.path.join(train_folder, f"X_train_{i}.npy")
            y_path = os.path.join(train_folder, f"y_train_{i}.npy")

            X_train = np.load(X_path)
            y_train = np.load(y_path)

            # COMPROBACIÓN DE NaNs
            if np.isnan(X_train).any() or np.isnan(y_train).any():
                print(f"❌ Chunk {i} tiene NaNs, lo saltamos.")
                continue

            # ESCALADO DE y_train
            y_train = y_train / 1000.0

            print(f"  ➤ Entrenando con chunk {i}... ({X_train.shape[0]} muestras)", end=' ')
            model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)
            print("✔️")

            del X_train, y_train

        except FileNotFoundError:
            print(f"⚠️ Chunk {i} no encontrado, saltando.")
            continue

# -------------------------------
# EVALUACIÓN RÁPIDA (chunk 0)
# -------------------------------
try:
    X_test = np.load(os.path.join(train_folder, "X_test_0.npy"))
    y_test = np.load(os.path.join(train_folder, "y_test_0.npy"))

    if np.isnan(X_test).any() or np.isnan(y_test).any():
        raise ValueError("❌ El set de test contiene NaNs.")

    y_test = y_test / 1000.0  # Escalar test igual que train

    print("\n📊 Evaluando modelo en test set (chunk 0)...")
    loss, mae = model.evaluate(X_test, y_test)
    print(f"🔍 Loss: {loss:.4f} | MAE: {mae:.4f}")

except Exception as e:
    print(f"⚠️ No se pudo evaluar el modelo: {e}")

# -------------------------------
# GUARDAR MODELO ENTRENADO
# -------------------------------
model_path = "../models/flight_price_predictor_global_eval.h5"
model.save(model_path)
print(f"✅ Modelo guardado en: {model_path}")

# -------------------------------
# EVALUACIÓN GLOBAL SOBRE TODOS LOS CHUNKS DE TEST
# -------------------------------
print("\n📊 Evaluando modelo con TODOS los chunks de test...")

total_loss = 0
total_mae = 0
total_samples = 0

for i in range(chunk_count):
    try:
        X_test = np.load(f"{train_folder}/X_test_{i}.npy")
        y_test = np.load(f"{train_folder}/y_test_{i}.npy")
        
        y_test = y_test / 1000.0  # desescalar
        loss, mae = model.evaluate(X_test, y_test, verbose=0)
        
        total_loss += loss * len(y_test)
        total_mae += mae * len(y_test)
        total_samples += len(y_test)

        print(f"✅ Chunk {i:02d}: MAE = {mae:.4f}, Samples = {len(y_test)}")

    except FileNotFoundError:
        print(f"⚠️ Chunk {i:02d} no encontrado, se omite.")
        continue

if total_samples > 0:
    avg_loss = total_loss / total_samples
    avg_mae = total_mae / total_samples
    print("\n📈 MÉTRICAS GLOBALES SOBRE TODO EL TEST SET")
    print(f"🔹 Loss promedio: {avg_loss:.6f}")
    print(f"🔹 MAE promedio:  {avg_mae:.4f} (×1000 = {avg_mae * 1000:.2f} €)")
else:
    print("❌ No se encontraron datos de test para evaluar.")


🚀 Entrenando el modelo por lotes...

🌀 Época 1/10
  ➤ Entrenando con chunk 0... (762107 muestras) ✔️
  ➤ Entrenando con chunk 1... (759244 muestras) ✔️
  ➤ Entrenando con chunk 2... (735888 muestras) ✔️
  ➤ Entrenando con chunk 3... (735300 muestras) ✔️
  ➤ Entrenando con chunk 4... (735871 muestras) ✔️
  ➤ Entrenando con chunk 5... (737430 muestras) ✔️
  ➤ Entrenando con chunk 6... (736117 muestras) ✔️
  ➤ Entrenando con chunk 7... (736203 muestras) ✔️
  ➤ Entrenando con chunk 8... (734632 muestras) ✔️
  ➤ Entrenando con chunk 9... (737012 muestras) ✔️
  ➤ Entrenando con chunk 10... (738777 muestras) ✔️
  ➤ Entrenando con chunk 11... (740420 muestras) ✔️
  ➤ Entrenando con chunk 12... (755498 muestras) ✔️
  ➤ Entrenando con chunk 13... (734076 muestras) ✔️
  ➤ Entrenando con chunk 14... (615153 muestras) ✔️
  ➤ Entrenando con chunk 15... (714638 muestras) ✔️
  ➤ Entrenando con chunk 16... (746610 muestras) ✔️
  ➤ Entrenando con chunk 17... (745083 muestras) ✔️
  ➤ Entrenando con chunk

🔍 Loss: 0.0000 | MAE: 0.0011
✅ Modelo guardado en: ../models/flight_price_predictor_global_eval.h5

📊 Evaluando modelo con TODOS los chunks de test...
✅ Chunk 00: MAE = 0.0011, Samples = 190527
✅ Chunk 01: MAE = 0.0011, Samples = 189811
✅ Chunk 02: MAE = 0.0010, Samples = 183972
✅ Chunk 03: MAE = 0.0010, Samples = 183825
✅ Chunk 04: MAE = 0.0010, Samples = 183968
✅ Chunk 05: MAE = 0.0010, Samples = 184358
✅ Chunk 06: MAE = 0.0010, Samples = 184030
✅ Chunk 07: MAE = 0.0010, Samples = 184051
✅ Chunk 08: MAE = 0.0010, Samples = 183659
✅ Chunk 09: MAE = 0.0010, Samples = 184253
✅ Chunk 10: MAE = 0.0010, Samples = 184695
✅ Chunk 11: MAE = 0.0010, Samples = 185106
✅ Chunk 12: MAE = 0.0010, Samples = 188875
✅ Chunk 13: MAE = 0.0010, Samples = 183519
✅ Chunk 14: MAE = 0.0010, Samples = 153789
✅ Chunk 15: MAE = 0.0010, Samples = 178660
✅ Chunk 16: MAE = 0.0010, Samples = 186653
✅ Chunk 17: MAE = 0.0010, Samples = 186271
✅ Chunk 18: MAE = 0.0010, Samples = 186481
✅ Chunk 19: MAE = 0.0010, Sample

In [8]:
import numpy as np
import pandas as pd
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import joblib
import matplotlib.pyplot as plt


# -------------------------------
# 1. CARGAR Y FILTRAR LOS DATOS
# -------------------------------
chunks_path = "../data_subsets/final_dataset_chunks/final_chunk_*.parquet"
all_chunks = [pd.read_parquet(file) for file in sorted(glob.glob(chunks_path))]
df = pd.concat(all_chunks, ignore_index=True)

# Eliminar columnas no necesarias (como legId)
df = df.drop(columns=['legId'])

# Filtrar outliers en el precio
q1 = df['totalFare'].quantile(0.01)
q99 = df['totalFare'].quantile(0.99)
df = df[(df['totalFare'] >= q1) & (df['totalFare'] <= q99)]

# -------------------------------
# 2. SEPARAR VARIABLES Y ESCALAR
# -------------------------------
X = df.drop(columns=['totalFare'])
y = df['totalFare'] / 1000.0  # escalar objetivo

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Guardar el scaler
joblib.dump(scaler, "../models/robust_scaler.pkl")

# -------------------------------
# 3. DEFINICIÓN DEL MODELO CON REGULARIZACIÓN
# -------------------------------
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],),
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='mean_squared_error',
    metrics=['mae']
)

# -------------------------------
# 4. ENTRENAMIENTO CON VALIDACIÓN
# -------------------------------
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=256,
    verbose=1
)

# -------------------------------
# 5. EVALUACIÓN Y GUARDADO
# -------------------------------
loss, mae = model.evaluate(X_test, y_test)
print(f"\n✅ Evaluación final - Loss: {loss:.6f} | MAE: {mae:.4f} (×1000 = {mae * 1000:.2f} €)")

model.save("../models/flight_price_predictor_v3_regularized.h5")

# -------------------------------
# 6. GRÁFICA DE ENTRENAMIENTO
# -------------------------------
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Pérdida durante el entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Loss")
plt.grid(True)
plt.show()


ValueError: No objects to concatenate

In [10]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib

# -------------------------------
# DEFINICIÓN DEL MODELO
# -------------------------------
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(21,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='mean_squared_error',
    metrics=['mae']
)

# -------------------------------
# ENTRENAMIENTO POR LOTES
# -------------------------------
train_folder = "../data_subsets/train_test"
chunk_count = 56   # Ajusta si tienes más o menos
epochs = 10        # Número de épocas completas
batch_size = 256

print("🚀 Entrenando el modelo por lotes...")
for epoch in range(epochs):
    print(f"\n🌀 Época {epoch+1}/{epochs}")
    for i in range(chunk_count):
        try:
            X_path = os.path.join(train_folder, f"X_train_{i}.npy")
            y_path = os.path.join(train_folder, f"y_train_{i}.npy")

            X_train = np.load(X_path)
            y_train = np.load(y_path)

            if np.isnan(X_train).any() or np.isnan(y_train).any():
                print(f"❌ Chunk {i} tiene NaNs, lo saltamos.")
                continue

            y_train = y_train / 1000.0  # Escalar si es necesario

            print(f"  ➤ Entrenando con chunk {i}... ({X_train.shape[0]} muestras)", end=' ')
            model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)
            print("✔️")

            del X_train, y_train

        except FileNotFoundError:
            print(f"⚠️ Chunk {i} no encontrado, saltando.")
            continue

# -------------------------------
# EVALUACIÓN GLOBAL SOBRE TODOS LOS TEST
# -------------------------------
print("\n📊 Evaluando el modelo sobre todos los test chunks...")
maes = []
losses = []
samples_total = 0

for i in range(chunk_count):
    try:
        X_test = np.load(os.path.join(train_folder, f"X_test_{i}.npy"))
        y_test = np.load(os.path.join(train_folder, f"y_test_{i}.npy"))

        if np.isnan(X_test).any() or np.isnan(y_test).any():
            print(f"❌ Chunk {i} tiene NaNs, lo saltamos.")
            continue

        y_test = y_test / 1000.0  # Escalar si fue escalado igual que train
        loss, mae = model.evaluate(X_test, y_test, verbose=0)

        maes.append(mae * X_test.shape[0])
        losses.append(loss * X_test.shape[0])
        samples_total += X_test.shape[0]

        print(f"✅ Chunk {i}: MAE = {mae:.4f}, Samples = {X_test.shape[0]}")

    except FileNotFoundError:
        continue

# -------------------------------
# RESULTADO GLOBAL PROMEDIO
# -------------------------------
if samples_total > 0:
    avg_mae = sum(maes) / samples_total
    avg_loss = sum(losses) / samples_total
    print("\n📌 MÉTRICAS GLOBALES SOBRE TODO EL TEST SET")
    print(f"🔷 Loss promedio: {avg_loss:.6f}")
    print(f"🔷 MAE promedio: {avg_mae:.4f} (×1000 = {avg_mae * 1000:.2f} €)")
else:
    print("⚠️ No se pudo evaluar el modelo: ningún chunk válido.")

# -------------------------------
# GUARDAR MODELO
# -------------------------------
model_path = "../models/flight_price_predictor_v3_clean_scaled.h5"
model.save(model_path)
print(f"\n💾 Modelo guardado en: {model_path}")


🚀 Entrenando el modelo por lotes...

🌀 Época 1/10
❌ Chunk 0 tiene NaNs, lo saltamos.
❌ Chunk 1 tiene NaNs, lo saltamos.
❌ Chunk 2 tiene NaNs, lo saltamos.
❌ Chunk 3 tiene NaNs, lo saltamos.
❌ Chunk 4 tiene NaNs, lo saltamos.
❌ Chunk 5 tiene NaNs, lo saltamos.
❌ Chunk 6 tiene NaNs, lo saltamos.
❌ Chunk 7 tiene NaNs, lo saltamos.
❌ Chunk 8 tiene NaNs, lo saltamos.
❌ Chunk 9 tiene NaNs, lo saltamos.
❌ Chunk 10 tiene NaNs, lo saltamos.
❌ Chunk 11 tiene NaNs, lo saltamos.
❌ Chunk 12 tiene NaNs, lo saltamos.
❌ Chunk 13 tiene NaNs, lo saltamos.
❌ Chunk 14 tiene NaNs, lo saltamos.
❌ Chunk 15 tiene NaNs, lo saltamos.
❌ Chunk 16 tiene NaNs, lo saltamos.
❌ Chunk 17 tiene NaNs, lo saltamos.
❌ Chunk 18 tiene NaNs, lo saltamos.
❌ Chunk 19 tiene NaNs, lo saltamos.
❌ Chunk 20 tiene NaNs, lo saltamos.
❌ Chunk 21 tiene NaNs, lo saltamos.
❌ Chunk 22 tiene NaNs, lo saltamos.
❌ Chunk 23 tiene NaNs, lo saltamos.
❌ Chunk 24 tiene NaNs, lo saltamos.
❌ Chunk 25 tiene NaNs, lo saltamos.
❌ Chunk 26 tiene NaNs, l

❌ Chunk 55 tiene NaNs, lo saltamos.
⚠️ No se pudo evaluar el modelo: ningún chunk válido.

💾 Modelo guardado en: ../models/flight_price_predictor_v3_clean_scaled.h5


In [2]:
print("Shape de X_train:", X_train.shape)
print("Shape de y_train:", y_train.shape)


Shape de X_train: (838860, 21)
Shape de y_train: (838860,)


In [9]:
import os
print(os.getcwd())

/home/cosmopato/FlightPriceProject/notebooks
